# Predictive Maintenance Inference - Batch or serverless real-time


With AutoML, our best model was automatically saved in our MLFlow registry.

All we need to do now is use this model to run Inferences. A simple solution is to share the model name to our Data Engineering team and they'll be able to call this model within the pipeline they maintained. That's what we did in our Delta Live Table pipeline!

Alternatively, this can be schedule in a separate job. Here is an example to show you how MLFlow can be directly used to retriver the model and run inferences.

*Make sure you run the previous notebook to be able to access the data.*

## Environment Recreation
The cell below downloads the model artifacts associated with your model in the remote registry, which include `conda.yaml` and `requirements.txt` files. In this notebook, `pip` is used to reinstall dependencies by default.

<!-- Collect usage data (view). Remove it to disable collection. View README for more details.  -->
<img width="1px" src="https://ppxrzfxige.execute-api.us-west-2.amazonaws.com/v1/analytics?category=lakehouse&org_id=4003492105941350&notebook=%2F04-Data-Science-ML%2F04.3-running-inference-iot-turbine&demo_name=lakehouse-iot-platform&event=VIEW&path=%2F_dbdemos%2Flakehouse%2Flakehouse-iot-platform%2F04-Data-Science-ML%2F04.3-running-inference-iot-turbine&version=1">

In [0]:
%pip install mlflow==2.22.0 databricks-sdk==0.40.0

In [0]:
%run ../_resources/00-setup $reset_all_data=false

##Deploying the model for batch inferences

<img style="float: right; margin-left: 20px" width="800" src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/retail/lakehouse-churn/ep_model_serving_creation.gif?raw=true" />

Now that our model is available in the Registry, we can load it to compute our inferences and save them in a table to start building dashboards.

We will use MLFlow function to load a pyspark UDF and distribute our inference in the entire cluster. If the data is small, we can also load the model with plain python and use a pandas Dataframe.

If you don't know how to start, Databricks can generate a batch inference notebook in just one click from the model registry: Open MLFlow model registry and click the "User model for inference" button!

### Scaling inferences using Spark 
We'll first see how it can be loaded as a spark UDF and called directly in a SQL function:

In [0]:
import mlflow
mlflow.set_registry_uri('databricks-uc')
#                                                                                                    Stage/version  
#                                                                                       Model name         |        
#                                                                                           |              |        
predict_maintenance = mlflow.pyfunc.spark_udf(spark, f"models:/{catalog}.{db}.dbdemos_turbine_maintenance@prod", "string", env_manager='virtualenv')
#We can use the function in SQL
spark.udf.register("predict_maintenance", predict_maintenance)
columns = predict_maintenance.metadata.get_input_schema().input_names()

In [0]:
columns = predict_maintenance.metadata.get_input_schema().input_names()
spark.table('turbine_hourly_features').withColumn("dbdemos_turbine_maintenance", predict_maintenance(*columns)).display()

In [0]:
%sql
SELECT turbine_id, predict_maintenance(hourly_timestamp, avg_energy, std_sensor_A, std_sensor_B, std_sensor_C, std_sensor_D, std_sensor_E, std_sensor_F, location, model, state) as prediction FROM turbine_hourly_features

### Pure pandas inference
If we have a small dataset, we can also compute our segment using a single node and pandas API:

In [0]:
from mlflow.store.artifact.models_artifact_repo import ModelsArtifactRepository
mlflow.set_registry_uri('databricks-uc')
local_path = ModelsArtifactRepository(f"models:/{catalog}.{db}.dbdemos_turbine_maintenance@prod").download_artifacts("") # download model from remote registry

requirements_path = os.path.join(local_path, "requirements.txt")
if not os.path.exists(requirements_path):
  dbutils.fs.put("file:" + requirements_path, "", True)

In [0]:
%pip install -r $requirements_path
dbutils.library.restartPython()

In [0]:
%run ../_resources/00-setup $reset_all_data=false

In [0]:
import mlflow
mlflow.set_registry_uri('databricks-uc')
model = mlflow.pyfunc.load_model(f"models:/{catalog}.{db}.dbdemos_turbine_maintenance@prod")
columns = model.metadata.get_input_schema().input_names()
df = spark.table('turbine_hourly_features').select(*columns).limit(10).toPandas()
df['churn_prediction'] = model.predict(df)
display(df)


## Realtime model serving with Databricks serverless serving

<img style="float: right; margin-left: 20px" width="800" src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/retail/lakehouse-churn/ep_model_serving_creation.gif?raw=true" />

Databricks also provides serverless serving.

Click on model Serving, enable realtime serverless and your endpoint will be created, providing serving over REST api within a Click.

Databricks Serverless offer autoscaling, including downscaling to zero when you don't have any traffic to offer best-in-class TCO while keeping low-latencies model serving.

## Real time model inference

Let's now deploy our model behind a realtime model serving endpoint.

We'll then use this endpoint in our GenAI Agentic demo to be able to fetch a turbine status in realtime


In [0]:
from mlflow.deployments import get_deploy_client

client = get_deploy_client("databricks")
try:
    endpoint = client.create_endpoint(
        name=MODEL_SERVING_ENDPOINT_NAME,
        config={
            "served_entities": [
                {
                    "name": "iot-maintenance-serving-endpoint",
                    "entity_name": f"{catalog}.{db}.{model_name}",
                    "entity_version": get_last_model_version(f"{catalog}.{db}.{model_name}"),
                    "workload_size": "Small",
                    "scale_to_zero_enabled": True
                }
            ]
        }
    )
except Exception as e:
    if "already exists" in str(e):
        print(f"Endpoint {catalog}.{db}.{MODEL_SERVING_ENDPOINT_NAME} already exists. Skipping creation.")
    else:
        raise e

while client.get_endpoint(MODEL_SERVING_ENDPOINT_NAME)['state']['config_update'] == 'IN_PROGRESS':
    time.sleep(10)

You can now view the status of the Feature Serving Endpoint in the table on the **Serving endpoints** page. Click **Serving** in the sidebar to display the page.

In [0]:
from mlflow import deployments


def score_model(dataset):
  client = mlflow.deployments.get_deploy_client("databricks")
  predictions = client.predict(endpoint=MODEL_SERVING_ENDPOINT_NAME, inputs=dataset.to_dict(orient='split'))

dataset = spark.table(f'turbine_hourly_features').select(*columns).limit(3).toPandas()
#Deploy your model and uncomment to run your inferences live!
#score_model(dataset)


# Next step: Leverage inferences and automate action to lower cost

## Automate action to react on potential turbine failure

We now have an end 2 end data pipeline analizing and predicting churn. We can now easily trigger actions to reduce downtime, such as dispatching a team earlier to fix the issue before an actual outage!

## Track windturbine failure and impact

Of course, this prediction can be re-used in our dashboard to analyse future failure and measure impact. 

<img width="800px" src="https://github.com/databricks-demos/dbdemos-resources/raw/main/images/manufacturing/lakehouse-iot-turbine/lakehouse-manuf-iot-dashboard-2.png">

<a dbdemos-dashboard-id="turbine-predictive" href="/sql/dashboardsv3/01f0624023b417aa91a2bab00ec4d4ba">Open the Predictive Maintenance AI/BI dashboard</a> | [Go back to the introduction]($../00-IOT-wind-turbine-introduction-DI-platform)